# New component

Fill with your own text

In [ ]:
%%writefile Predictor.py
import logging
from typing import List, Iterable, Union

import numpy as np
import pandas as pd
from platiagro import load_model

logger = logging.getLogger(__name__)


class Predictor(object):
    """
    Model template. You can load your model parameters in __init__ from a location accessible at runtime.
    """

    def __init__(self, dataset=None, target=None, experiment_id=None):
        """
        Add any initialization parameters. These will be passed at runtime from the graph definition parameters defined in your seldondeployment kubernetes resource manifest.
        """
        logger.info("Initializing")

        # Loads a model
        model = load_model(name=experiment_id)

        logger.info("Init complete!")

    def predict(self, X: np.ndarray, feature_names: Iterable[str]) -> Union[np.ndarray, List, str, bytes]:
        """Returns a prediction.

        Args:
            X (numpy.array): Array-like with data.
            feature_names (iterable, optional): Array of feature names.

        Returns:
            numpy.array: Array-like with predictions.
        """
        logger.info("Predict called - will run predictions")

        # Builds a DataFrame from numpy.array.
        df = pd.DataFrame(X, columns=feature_names)

        X_predict = df.to_numpy()
        
        # start your code here...

        return X_predict

## Deployment Test

It simulates a model deployed by PlatIAgro

In [ ]:
%%sh
MODEL_NAME=Predictor
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0
LOG_LEVEL=DEBUG
PARAMETERS='[{"type":"STRING","name":"dataset","value":"iris"},{"type":"STRING","name":"target","value":"col4"},{"type":"STRING","name":"experiment_id","value":"99284308-cd3f-47d4-ab71-9c57acbb4d7b"}]'

seldon-core-microservice $MODEL_NAME $API_TYPE \
    --service-type $SERVICE_TYPE \
    --persistence $PERSISTENCE \
    --parameters $PARAMETERS \
    --log-level $LOG_LEVEL > log.txt 2>&1 &

## Make predictions

In [ ]:
!curl localhost:5000/predict -d 'json={"data":{"names":["col0", "col1", "col2", "col3", "col4", "col5", "col6", "col7", "col8", "col9", "col10", "col11", "col12"], "ndarray":[[0.00632, 18.0, 2.31, 0, 0.5379999999999999, 6.575, 65.2, 4.09, 1, 296, 15.3, 396.9, 4.98]]}}'

## View logs

In [ ]:
!cat log.txt

## Clean up the test

In [ ]:
!ps -ef | grep [s]eldon-core-microservice | awk '{print $2}' | xargs -r kill